# Outbreak Prediction Evaluation
## 7-Day Risk Forecast & Alert System

**Components**:
- Outbreak risk timeline visualization
- Spatial risk heatmaps
- Alert generation and summary
- Performance metrics (MAE target < 0.15)

In [ ]:
import sys
sys.path.insert(0, '../..')

import numpy as np
import matplotlib.pyplot as plt

from src.data.iot_simulator import IoTSimulator
from src.utils.alert_system import AlertGenerator
from src.visualization.outbreak_plots import (
    plot_risk_timeline, plot_spatial_heatmap,
    plot_temporal_progression, plot_alert_summary
)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 1. Simulate Outbreak Scenarios

In [ ]:
# Generate IoT data for multiple fields
config = {
    'num_fields': 10,
    'days': 365,
    'seed': 42,
    'temperature': {'base_mean': 25, 'base_amplitude': 8, 'diurnal_amplitude': 5, 'ar1_coeff': 0.7, 'noise_std': 1.5},
    'humidity': {'base_mean': 70, 'anti_corr_factor': -1.5, 'noise_std': 5},
    'rain': {'zero_inflation': 0.7, 'gamma_shape': 0.8, 'gamma_scale': 5.0},
    'soil_moisture': {'initial': 0.35, 'rain_factor': 0.05, 'decay_rate': 0.03},
    'wind': {'log_mean': 1.5, 'log_std': 0.6},
    'disease_seir': {
        'S0': 0.95, 'E0': 0.03, 'I0': 0.02, 'R0': 0.0,
        'beta_base': 0.3, 'sigma': 0.1, 'gamma': 0.05,
        'temp_optimal': 25.0, 'humidity_optimal': 85.0,
    },
}

sim = IoTSimulator(config, seed=42)
all_data = sim.generate_all()
print(f'Generated data for {len(all_data)} fields')

## 2. Disease Progression Visualization

In [ ]:
df = all_data[0]
plot_temporal_progression(
    dates=df['date'].dt.strftime('%Y-%m-%d').tolist(),
    disease_prevalence=df['disease_prevalence'].tolist(),
    temperature=df['temperature'].tolist(),
    humidity=df['humidity'].tolist(),
    title='Field 0 - Disease Progression with Environmental Factors',
)

## 3. Simulated 7-Day Risk Forecasts

In [ ]:
# Simulate model predictions for several fields
np.random.seed(42)

for field_id in [0, 3, 7]:
    # Use last 7 days of disease prevalence as simulated prediction
    df = all_data[field_id]
    risk_7d = df['disease_prevalence'].iloc[-7:].values.tolist()
    # Scale to be more realistic as predictions
    risk_7d = [min(1.0, r * 3 + np.random.uniform(0, 0.1)) for r in risk_7d]

    plot_risk_timeline(
        risk_7d=risk_7d,
        field_id=str(field_id),
        threshold_high=0.75,
        threshold_medium=0.5,
    )

## 4. Spatial Risk Distribution

In [ ]:
# Generate field locations and max risk values
rng = np.random.default_rng(42)
field_locations = {}
risk_values = {}

for fid in range(10):
    lat = -23.0 + rng.uniform(-1, 1)
    lon = -49.0 + rng.uniform(-1, 1)
    field_locations[str(fid)] = (lat, lon)
    risk_values[str(fid)] = float(all_data[fid]['disease_prevalence'].max())

plot_spatial_heatmap(
    field_locations=field_locations,
    risk_values=risk_values,
    title='Spatial Risk Distribution Across Fields',
)

## 5. Alert Generation

In [ ]:
class_names = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
    'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight',
    'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy',
    'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
    'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
]

generator = AlertGenerator(class_names=class_names)

# Generate sample alerts
alerts = []
for fid in range(10):
    # Simulated predictions
    disease_probs = np.random.dirichlet(np.ones(15)).tolist()
    risk_7d = np.clip(all_data[fid]['disease_prevalence'].iloc[-7:].values * 3, 0, 1).tolist()
    severity_probs = np.random.dirichlet(np.ones(4)).tolist()

    alert = generator.generate_alert(
        field_id=f'field_{fid:03d}',
        disease_probs=disease_probs,
        outbreak_risk_7d=risk_7d,
        severity_probs=severity_probs,
    )
    alerts.append(alert)

print(f'Generated {len(alerts)} alerts')
print(f'\nSample alert:')
print(alerts[0].to_json())

In [ ]:
# Alert summary
alert_dicts = [a.to_dict() for a in alerts]
plot_alert_summary(alert_dicts)

# Filter high-risk alerts
high_risk = generator.filter_alerts(alerts, min_risk_level='HIGH')
print(f'\nHigh-risk alerts: {len(high_risk)}')
for a in high_risk:
    print(f'  Field {a.field_id}: {a.disease_name} ({a.risk_level})')